In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import random
import matplotlib.pyplot as plt
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

In [ ]:
!unzip /content/gdrive/MyDrive/DatasetV2Trimmed5.zip -d /content/dataset

In [ ]:
base_dir="/content/dataset"
os.listdir('/content/dataset')

In [ ]:
IMAGE_SIZE=224
BATCH_SIZE=50

#Perform augmentation on the images
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    rotation_range=40,
    shear_range=0.2,
    horizontal_flip=True,
)

test_datagen=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

train_datagen=train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training'
)

test_datagen=test_datagen.flow_from_directory(
    base_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='validation'

)


In [ ]:
class_indices = train_datagen.class_indices
class_labels = list(class_indices.keys())

print("Class Indices:", class_indices)
print("Class Labels:", class_labels)

In [ ]:
#Will do transfer learning from MobileNetV2
base = tf.keras.applications.mobilenet_v2.MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3), pooling='avg')
base.trainable = False
model = tf.keras.models.Sequential([
    # tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    # tf.keras.layers.MaxPooling2D((2, 2)),
    # tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    # tf.keras.layers.MaxPooling2D((2, 2)),
    # tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    # tf.keras.layers.MaxPooling2D((2, 2)),
    # tf.keras.layers.Flatten(),
    base,
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(5, activation='softmax') #Change this according to number of classes
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
#Set the callbacks
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    'model_checkpoint,h5',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    verbose=1
)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    restore_best_weights=True,
)

In [ ]:
#Train the model
history = model.fit(
    train_datagen,
    epochs=15,
    validation_data=test_datagen,
    callbacks=[checkpoint,early_stopping])

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Performance')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'])
plt.show()


In [ ]:
uploaded = files.upload()

class_labels = ['Glass_ClearGlass', 'Metal_AluminiumCans',
                'Paper_Paper', 'Plastic_PET','Textiles_Textiles']

'''class_labels = ['AHP_AHP', 'Composite_CompositeFnBCarton', 'Composite_Matresses',
                'Glass_BrownGlass', 'Glass_ClearGlass', 'Glass_GreenGlass',
                'IronMetal_StainlessSteel', 'IronMetal_Steel', 'Metal_AluminiumCans',
                'NonIronMetal_AlumuniumNonCan', 'NonIronMetal_Wiring', 'Paper_Cardboard',
                'Paper_Paper', 'Plastic_HDPEM', 'Plastic_PET', 'Textiles_Footwear',
                'Textiles_Textiles'] '''#Original complete version

for fn in uploaded.keys():
    # Predicting images
    path = fn
    img = load_img(path, target_size=(224, 224))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    predictions = model.predict(images, batch_size=10)
    predicted_class_index = np.argmax(predictions[0])
    predicted_label = class_labels[predicted_class_index]

    print("Filename:", fn)
    print("Predicted Class:", predicted_label)
    print("Prediction Scores:", predictions[0])

In [ ]:
#generate saved model
export_dir = 'saved_model/1'

tf.saved_model.save(model, export_dir)

In [ ]:
model.save('model.h5')

In [ ]:
#convert savedmodel to TFLite
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)

converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model = converter.convert()

In [ ]:
import pathlib

tflite_model_file = pathlib.Path('./model.tflite')
tflite_model_file.write_bytes(tflite_model)

In [ ]:
#download TFLite model
try:
    from google.colab import files
    files.download(tflite_model_file)
except:
    pass